In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras_spiking
import tensorflow as tf

In [2]:
# Load CIFAR 10
(
    (train_images, train_labels),
    (test_images, test_labels),
) = tf.keras.datasets.mnist.load_data()

In [3]:
# normalize images so values are between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

In [4]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

topologies = [
    [32],
    [32, 32],
    [64, 32],
    [64],
    [64, 64],
    [128, 64],
    [128],
    [128, 128],
    [256, 128]
]

def generate_non_spiking_model(topology):
    layers = [tf.keras.layers.Flatten(input_shape=(28, 28))]
    for hidden_layer_size in topology:
        layers.append(tf.keras.layers.Dense(hidden_layer_size, activation="relu"))
    layers.append(tf.keras.layers.Dense(10))
    model = tf.keras.Sequential(layers)
    return model

def generate_spiking_model(topology):
    layers = [tf.keras.layers.Reshape((-1, 28 * 28), input_shape=(None, 28, 28))]
    for hidden_layer_size in topology:
        layers.append(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(hidden_layer_size)))
        layers.append(keras_spiking.SpikingActivation("relu", spiking_aware_training=False))
        layers.append(tf.keras.layers.GlobalAveragePooling1D(),)
    layers.append(tf.keras.layers.Dense(10))
    model = tf.keras.Sequential(layers)
    return model

In [5]:
import numpy as np
np.shape(test_images)

(10000, 28, 28)

In [6]:
def train(input_model, train_x, test_x):
    print(np.shape(test_x))
    print(np.shape(test_x[0]))
    assert len(test_x) == len(test_labels)
    validation_data = []
    for i in range(len(test_x)):
        validation_data.append((test_x[i], test_labels[i]))
    
    input_model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    input_model.fit(train_x, train_labels, epochs=10, callbacks=[es])

    _, test_acc = input_model.evaluate(test_x, test_labels, verbose=2)

    print("\nTest accuracy:", test_acc)

spiking_mod_1 = generate_spiking_model([128])
n_steps = 10
train_sequences = np.tile(train_images[:, None], (1, n_steps, 1, 1))
test_sequences = np.tile(test_images[:, None], (1, n_steps, 1, 1))
train(spiking_mod_1, train_sequences, test_sequences)

(10000, 10, 28, 28)
(10, 28, 28)
Epoch 1/10
1875/1875 [==============================] - 31s 15ms/step - loss: 0.4276 - accuracy: 0.8789
Epoch 2/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.1140 - accuracy: 0.9660
Epoch 3/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0733 - accuracy: 0.9781
Epoch 4/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.0524 - accuracy: 0.9844
Epoch 5/10
1340/1875 [====================>.........] - ETA: 7s - loss: 0.0368 - accuracy: 0.9891

KeyboardInterrupt: 